In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
dataset_path="/content/drive/MyDrive/MRP/dataset/HI-Small_Trans.csv"

df=pd.read_csv(dataset_path)
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['date'] = df['Timestamp'].dt.date
df['hour'] = df['Timestamp'].dt.hour
df['day'] = df['Timestamp'].dt.day
df['weekday'] = df['Timestamp'].dt.day_name()
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,date,hour,day,weekday
0,2022-09-01 00:20:00,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,2022-09-01,0,1,Thursday
1,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,2022-09-01,0,1,Thursday
2,2022-09-01 00:00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,2022-09-01,0,1,Thursday
3,2022-09-01 00:02:00,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,2022-09-01,0,1,Thursday
4,2022-09-01 00:06:00,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,2022-09-01,0,1,Thursday


In [ ]:
df = df.drop(columns=['Timestamp'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Label encode categorical columns
for col in ['Account', 'Account.1', 'Payment Format','Receiving Currency','Payment Currency','weekday','date', 'From Bank', 'To Bank']:
    df[col] = LabelEncoder().fit_transform(df[col])

In [27]:
df.head()
df_sample=df.sample(n=100000, random_state=42)

In [28]:
df_sample.describe()

,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,date,hour,day,weekday
count,100000.000000,100000.000000,100000.000000,100000.00000,1.000000e+05,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,1952.921670,160913.818340,1654.033260,207841.96273,4.190104e+06,8.363410,3.917982e+06,8.391980,3.047090,0.001050,3.845500,10.466940,4.845500,2.672800
std,3598.573478,145934.179843,1812.648728,122202.21727,3.585235e+08,4.117768,3.575921e+08,4.118167,1.487829,0.032387,3.075351,7.349205,3.075351,2.101605
min,0.000000,0.000000,0.000000,0.00000,1.000000e-06,0.000000,1.000000e-06,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,33.000000,28154.500000,340.000000,101861.00000,1.823575e+02,4.000000,1.836875e+02,4.000000,3.000000,0.000000,1.000000,4.000000,2.000000,0.000000
50%,595.000000,119922.000000,637.000000,206273.00000,1.408190e+03,10.000000,1.410125e+03,10.000000,3.000000,0.000000,4.000000,10.000000,5.000000,4.000000
75%,970.000000,271894.000000,3652.000000,313860.25000,1.212346e+04,12.000000,1.208956e+04,12.000000,4.000000,0.000000,7.000000,17.000000,8.000000,4.000000
max,30449.000000,496993.000000,15760.000000,420634.00000,1.079558e+11,14.000000,1.079558e+11,14.000000,6.000000,1.000000,17.000000,23.000000,18.000000,6.000000


In [29]:
#Prepare features and labels
X = df_sample.drop(columns=['Is Laundering'])
y = df_sample['Is Laundering']
df_sample.head()

,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,date,hour,day,weekday
298872,31,349994,3385,321293,4981.60,10,4981.60,10,3,0,0,0,1,4
746726,8,2283,663,21319,297.72,12,297.72,12,3,0,0,13,1,4
405190,993,311083,793,260536,32.90,14,32.90,14,5,0,0,2,1,4
1388703,8,120832,485,115202,194634.45,7,194634.45,7,3,0,1,8,2,0
4713645,723,255317,521,246702,698940.91,12,698940.91,12,0,0,8,18,9,0


## Bootstrap Evaluation

In [39]:
from sklearn.utils import resample
from sklearn.metrics import f1_score, accuracy_score, recall_score
import numpy as np

def bootstrap_evaluation(model, X, y, n_iterations=100, test_size=0.3, random_state=42):
    np.random.seed(random_state)
    metrics = {
        'accuracy': [],
        'f1_minority': [],
        'recall_minority': [],
        'macro_f1': [],
    }

    for i in range(n_iterations):
        # Sample with replacement
        X_sample, y_sample = resample(X, y, replace=True, stratify=y)

        # Train-test split within the sample
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X_sample, y_sample, test_size=test_size, stratify=y_sample, random_state=i
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Identify the minority class (assumes binary classification)
        classes, counts = np.unique(y, return_counts=True)
        minority_class = classes[np.argmin(counts)]

        metrics['accuracy'].append(accuracy_score(y_test, y_pred))
        metrics['f1_minority'].append(f1_score(y_test, y_pred, pos_label=minority_class))
        metrics['recall_minority'].append(recall_score(y_test, y_pred, pos_label=minority_class))
        metrics['macro_f1'].append(f1_score(y_test, y_pred, average='macro'))

    return metrics


In [33]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

logreg = LogisticRegression(max_iter=1000,class_weight='balanced')
xgb_model = xgb.XGBClassifier()
lgb_model = lgb.LGBMClassifier()

metrics_logreg = bootstrap_evaluation(logreg, X, y)
# metrics_xgb = bootstrap_evaluation(xgb_model, X, y)
# metrics_lgb = bootstrap_evaluation(lgb_model, X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [34]:
metrics_logreg

{'accuracy': [0.692,
  0.5292,
  0.7999333333333334,
  0.8718333333333333,
  0.6684,
  0.7160333333333333,
  0.6881333333333334,
  0.6695666666666666,
  0.6323666666666666,
  0.8645,
  0.7014,
  0.8166333333333333,
  0.6616333333333333,
  0.6941333333333334,
  0.6957,
  0.8153,
  0.7139666666666666,
  0.7203666666666667,
  0.6773333333333333,
  0.8268,
  0.7680666666666667,
  0.8448666666666667,
  0.5635333333333333,
  0.691,
  0.7017666666666666,
  0.6966,
  0.7107,
  0.7506666666666667,
  0.7458333333333333,
  0.8345666666666667],
 'f1_minority': [0.004524886877828055,
  0.003105590062111801,
  0.006949040370615486,
  0.01334359763920965,
  0.0026067776218167233,
  0.0037422523681440766,
  0.00531575590048905,
  0.0026159573397726127,
  0.003433631517122978,
  0.012630556230264756,
  0.004002668445630421,
  0.00972097209720972,
  0.0033382425135002453,
  0.004340277777777778,
  0.005663871038013289,
  0.0096514745308311,
  0.004639832966013223,
  0.003563368571089203,
  0.00411522633

In [40]:
metrics_xgb = bootstrap_evaluation(xgb_model, X, y)
metrics_lgb = bootstrap_evaluation(lgb_model, X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 74, number of negative: 69926
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1625
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001057 -> initscore=-6.851128
[LightGBM] [Info] Start training from score -6.851128
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 73, number of negative: 69927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wis

In [41]:
# metrics_xgb = bootstrap_evaluation(xgb_model, X, y)
# metrics_lgb = bootstrap_evaluation(lgb_model, X, y)
print("metrics_xgb:",metrics_xgb)
print("metrics_lgb:",metrics_lgb)


metrics_xgb: {'accuracy': [0.9995, 0.9996, 0.9994, 0.9994666666666666, 0.9994, 0.9994666666666666, 0.9994666666666666, 0.9994, 0.9994666666666666, 0.9994333333333333, 0.9994666666666666, 0.9993, 0.9998, 0.9995666666666667, 0.9993666666666666, 0.9994333333333333, 0.9994666666666666, 0.9994, 0.9995333333333334, 0.9994666666666666, 0.9992666666666666, 0.9992333333333333, 0.9995333333333334, 0.9997666666666667, 0.9994, 0.9992666666666666, 0.9993666666666666, 0.9994666666666666, 0.9995666666666667, 0.9993, 0.9995, 0.9994, 0.9994666666666666, 0.9994333333333333, 0.9995333333333334, 0.9993666666666666, 0.9996333333333334, 0.9994666666666666, 0.9995, 0.9994, 0.9995, 0.9994666666666666, 0.9995, 0.9995333333333334, 0.9994666666666666, 0.9995333333333334, 0.9994, 0.9996, 0.9994666666666666, 0.9996333333333334, 0.9995, 0.9995666666666667, 0.9996, 0.9993666666666666, 0.9995666666666667, 0.9991333333333333, 0.9994333333333333, 0.9995333333333334, 0.9994333333333333, 0.9995, 0.9994666666666666, 0.999

In [42]:
def confidence_interval(data, confidence=0.95):
    lower = np.percentile(data, (1 - confidence) / 2 * 100)
    upper = np.percentile(data, (1 + confidence) / 2 * 100)
    return lower, upper

for name, metrics in zip(['LogReg', 'LightGBM', 'XGBoost'],
                         [metrics_logreg, metrics_lgb, metrics_xgb]):
    print(f"\n{name} Confidence Intervals:")
    for metric_name, scores in metrics.items():
        lower, upper = confidence_interval(scores)
        print(f"  {metric_name}: {np.mean(scores):.3f} [{lower:.3f}, {upper:.3f}]")


LogReg Confidence Intervals:
  accuracy: 0.725 [0.554, 0.867]
  f1_minority: 0.006 [0.003, 0.013]
  recall_minority: 0.671 [0.416, 0.892]
  macro_f1: 0.422 [0.358, 0.471]

LightGBM Confidence Intervals:
  accuracy: 0.996 [0.994, 0.999]
  f1_minority: 0.200 [0.055, 0.552]
  recall_minority: 0.401 [0.129, 0.699]
  macro_f1: 0.599 [0.526, 0.776]

XGBoost Confidence Intervals:
  accuracy: 0.999 [0.999, 1.000]
  f1_minority: 0.664 [0.488, 0.823]
  recall_minority: 0.514 [0.333, 0.699]
  macro_f1: 0.832 [0.744, 0.911]


In [45]:
from scipy.stats import ttest_rel

# Example: Compare macro F1 between XGBoost and LightGBM
stat, p = ttest_rel(metrics_xgb['macro_f1'], metrics_lgb['macro_f1'])
print(f"XGBoost vs LightGBM macro F1 - p-value: {p:.5f}")

XGBoost vs LightGBM macro F1 - p-value: 0.00000
